# Analiza Spotify Podataka - Projekat 2 - Osnovi operativnih sistema

## Uvod

Ovaj projektni zadatak ima za cilj analizu Spotify skupa podataka koristeći Spark DataFrame API.

### Pokretanje projekta

Za pokretanje ovog projekta potrebno je imati instaliran Python 3.12.3, Apache Spark, kao i biblioteke navedene ispod, kako bi se osiguralo da radi sve kao kod mene. Takodje, zbog problema sa pysparkom na windowsu, preporučuje se instalacija WSL-a, te pokretanje VSC u virtuelnom okruženju.

### Instalacija Spotify skupa podataka 

Da bi instalirali skup podataka na kojem se vrši analiza, pratiti upute sa linka: [Spotify skup podataka](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset)

### Instalacija potrebnih biblioteka

Za pokretanje ovog projekta, potrebno je instalirati sledeće biblioteke (dostupne putem pip):

- `pyspark`
- `pandas`

Možete instalirati ove biblioteke koristeći sledeću komandu u CMD-u:

```bash
pip install pyspark pandas

_______________________________________________________________________________________________________________________________________________________

## Učitavanje biblioteka i skupa podataka

In [1]:
import pandas as pd
from itertools import combinations
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    BooleanType,
    LongType,
    DoubleType,
    ArrayType
)

In [2]:
spark = SparkSession.builder.appName("Analiza Spotify Podataka - Projekat2 OPOS").getOrCreate()

# Radi bolje preglednosti nakon ovog bloka se neće ispisivati WARN poruke kojih je puno, nego samo ERRORI
spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
25/03/19 23:18:18 WARN Utils: Your hostname, DESKTOP-0THU24A resolves to a loopback address: 127.0.1.1; using 172.31.14.97 instead (on interface eth0)
25/03/19 23:18:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/19 23:18:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Provjera tipa podataka uz pomoć pandas biblioteke, isto se moglo uraditi i sa pysparkom prilikom učitavanja CSV-a sa inferSchema=True, pa onda printSchema, ali ovo mi je dalo preciznije rezultate.

In [3]:
df_pd = pd.read_csv('datasets/spotify.csv')

# Broj redova i kolona, kao i tipovi podataka za kolone
df_pd.shape,df_pd.dtypes   

((114000, 21),
 Unnamed: 0            int64
 track_id             object
 artists              object
 album_name           object
 track_name           object
 popularity            int64
 duration_ms           int64
 explicit               bool
 danceability        float64
 energy              float64
 key                   int64
 loudness            float64
 mode                  int64
 speechiness         float64
 acousticness        float64
 instrumentalness    float64
 liveness            float64
 valence             float64
 tempo               float64
 time_signature        int64
 track_genre          object
 dtype: object)

Definisanje tačne šeme podataka na osnovu informacija iznad i učitavanje skupa podataka

In [4]:
schema = StructType([
    StructField("", LongType(), True),  # Redni broj, nije potreban za ovu analizu
    StructField("track_id", StringType(), True),
    StructField("artists", StringType(), True),
    StructField("album_name", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("popularity", LongType(), True),  
    StructField("duration_ms", LongType(), True),  
    StructField("explicit", BooleanType(), True),
    StructField("danceability", DoubleType(), True),
    StructField("energy", DoubleType(), True),
    StructField("key", LongType(), True),  
    StructField("loudness", DoubleType(), True),
    StructField("mode", LongType(), True),  
    StructField("speechiness", DoubleType(), True),
    StructField("acousticness", DoubleType(), True),
    StructField("instrumentalness", DoubleType(), True),
    StructField("liveness", DoubleType(), True),
    StructField("valence", DoubleType(), True),
    StructField("tempo", DoubleType(), True),
    StructField("time_signature", LongType(), True),  
    StructField("track_genre", StringType(), True)
])

# Kod imena pjesme i albuma detektovao zareze, pa izbacivao da je neki broj zanr ili slicno
# Chatgpt kod help:
df = spark.read.option("quote", "\"").option("escape", "\"").csv("datasets/spotify.csv", header=True, schema=schema)

Validacija učitanih podataka

In [5]:
print(f"Broj redova i kolona: {df.count()}, {len(df.columns)}")
df.printSchema()
df.show(10)

Broj redova i kolona: 114000, 21
root
 |-- : long (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: long (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: long (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- track_genre: string (nullable = true)

+---+--------------------+--------------------+--------------------+--------------------+----------+-----------+-------

## Analiza distribucije podataka

Analiza distribucije numerickih kolona

In [ ]:
num_values_distribution=df.select(
    ["", "popularity", "duration_ms", "explicit", "danceability", "energy", "key", "loudness",
     "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"]).describe()
num_values_distribution.show()
num_values_distribution.write.format("json").mode("overwrite").save("statistika_number")

+-------+-----------------+-----------------+------------------+-------------------+------------------+-----------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|                 |       popularity|       duration_ms|       danceability|            energy|              key|          loudness|        speechiness|       acousticness|   instrumentalness|           liveness|            valence|             tempo|     time_signature|
+-------+-----------------+-----------------+------------------+-------------------+------------------+-----------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           114000|           114000|            114000|             114000|            114000|           114000|            114000|             114000|

Analiza distribucije tekstualnih kolona

In [ ]:
string_columns = ["track_id", "artists", "album_name", "track_name", "track_genre"]

string_stats = {}

for column in string_columns:
    result = df.groupBy(column).count()
    string_stats[column] = result.collect()

string_distribution_df = []
for column, rows in string_stats.items():
    for row in rows:
        string_distribution_df.append((column, row[column], row['count']))

df_string_distribution = spark.createDataFrame(string_distribution_df, ["column", "value", "count"])
df_string_distribution.write.format("json").mode("overwrite").save("statistika_string")

## Čišćenje podataka

In [6]:
# Brisanje nepotrebne kolone za redni broj, radi optimizacije
df=df.drop("")
print(f"Broj redova i kolona: {df.count()}, {len(df.columns)}")
# Broj kolona je sad za 1 manji u odnosu na početak kad je bilo 21 kolona

Broj redova i kolona: 114000, 20


In [7]:
# Iz analize distribucije sam zaključio da postoje pjesme duplikati, tj. dupli track_id redovi,
# njih ću obrisati i zadržati samo prvo pojavljivanje pjesme, radi bolje i jednake analize
df = df.dropDuplicates(["track_id"])
print(f"Broj redova i kolona: {df.count()}, {len(df.columns)}")
# Broj redova je sad manji za oko 24000, tako da će se sa ovim ubrzati analiza za 20%

Broj redova i kolona: 89741, 20


In [8]:
# Izbacivanje redova koji nemaju neku od vrijednosti, tj. neka kolona im je prazna, ukoliko postoje
df = df.na.drop()
print(f"Broj redova i kolona: {df.count()}, {len(df.columns)}")
# Iz ispisa se vidi da je broj redova za 1 manji, tako da je jedan red sadržavao neku NULL vrijednost
# Taj red sam izbacio da se ne zezne račun poslije na osnovu ovog reda

Broj redova i kolona: 89740, 20


In [9]:
# Provjera da li se ucitava dobro sve, jer je track genre bio nekad broj, ali poslije ucitavanja ispravnog je dobro sad
df.select("track_genre").distinct().show(truncate=False)


+-----------------+
|track_genre      |
+-----------------+
|anime            |
|singer-songwriter|
|folk             |
|hardstyle        |
|pop              |
|alternative      |
|death-metal      |
|detroit-techno   |
|idm              |
|k-pop            |
|j-dance          |
|ambient          |
|guitar           |
|goth             |
|cantopop         |
|blues            |
|study            |
|malay            |
|dance            |
|breakbeat        |
+-----------------+
only showing top 20 rows



## #1

Identifikovati 10 najčešćih kolaboracija između umjetnika (dva ili više umjetnika koji nastupaju zajedno na pjesmi) i izračunajte prosječnu popularnost njihovih pesama. Za prosječnu razliku popularnosti solo pjesama manje popularnog izvođača i prosječne popularnosti kolaboracije.

In [12]:
def create_artist_pairs(artists):
    if artists is None or len(artists) < 2:  # Provera da li su umetnici prisutni
        return []
    return [tuple(sorted(comb)) for comb in combinations(artists, 2)]

# Razdvajanje umetnika u listu koristeći ';'
df_with_artists = df.withColumn("artists", F.split(F.col("artists"), ";"))

# Filtriranje redova sa praznim ili None umetnicima
df_with_artists = df_with_artists.filter(F.size(F.col("artists")) > 0)

# 1. Filtriramo solo pesme - pesme sa samo jednim umetnikom
solo_songs = df_with_artists.filter(F.size(F.col("artists")) == 1)

# 2. Izračunavamo prosečnu popularnost za svakog umetnika u solo pesmama
solo_avg_popularity = solo_songs.select(F.explode("artists").alias("artist"), "popularity") \
    .groupBy("artist") \
    .agg(F.avg("popularity").alias("avg_solo_popularity"))
    

# UDF za kreiranje parova umetnika
create_pairs_udf = F.udf(create_artist_pairs, ArrayType(ArrayType(StringType())))

df_with_pairs = df_with_artists.withColumn("artist_pairs", create_pairs_udf(F.col("artists")))
    
collaboration_count = df_with_pairs.select(F.explode("artist_pairs").alias("artist_pair"), "popularity") \
    .groupBy("artist_pair") \
    .agg(F.count("artist_pair").alias("count"), F.avg("popularity").alias("avg_popularity")) \
    .orderBy(F.desc("count"))

# 3. Ekstrakcija pojedinačnih umetnika iz parova
collab = collaboration_count.withColumn("artist1", F.col("artist_pair")[0]) \
                            .withColumn("artist2", F.col("artist_pair")[1])

# 4. Spajanje solo popularnosti za oba umetnika
# Pripremite DataFrame za artist1
solo_artist1 = solo_avg_popularity.withColumnRenamed("artist", "artist1") \
                                  .withColumnRenamed("avg_solo_popularity", "artist1_solo_popularity")
# Pripremite DataFrame za artist2
solo_artist2 = solo_avg_popularity.withColumnRenamed("artist", "artist2") \
                                  .withColumnRenamed("avg_solo_popularity", "artist2_solo_popularity")

# Spojite ih sa tabelom kolaboracija
collab_joined = collab.join(solo_artist1, on="artist1", how="left") \
                      .join(solo_artist2, on="artist2", how="left")

# 5. Postavljanje vrednosti 0 ako umetnik nema solo pesme
collab_with_less = collab_joined.withColumn("artist1_solo_popularity", F.coalesce(F.col("artist1_solo_popularity"), F.lit(0))) \
                                .withColumn("artist2_solo_popularity", F.coalesce(F.col("artist2_solo_popularity"), F.lit(0)))

# 6. Određivanje manje popularnog umetnika na osnovu solo popularnosti
collab_with_less = collab_with_less.withColumn("less_popular_artist", 
    F.when(F.col("artist1_solo_popularity") < F.col("artist2_solo_popularity"), F.col("artist1"))
     .otherwise(F.col("artist2"))
).withColumn("less_popular_avg_popularity", 
    F.when(F.col("artist1_solo_popularity") < F.col("artist2_solo_popularity"), F.col("artist1_solo_popularity"))
     .otherwise(F.col("artist2_solo_popularity"))
)

# 7. Opcionalno: računanje razlike između prosečne popularnosti kolaboracija i solo popularnosti manje popularnog umetnika
collab_with_less = collab_with_less.withColumn("popularity_difference", 
    F.abs(F.col("avg_popularity") - F.col("less_popular_avg_popularity"))
)

# 8. Kreiranje čitljivog prikaza imena umetnika u paru
collab_with_less = collab_with_less.withColumn("artist_pair_names", 
    F.concat_ws(" & ", F.col("artist1"), F.col("artist2"))
)

# 9. Sortiranje rezultata po broju kolaboracija
result = collab_with_less.select("artist_pair_names", "count", "avg_popularity", 
                                   "less_popular_artist", "less_popular_avg_popularity", "popularity_difference") \
                          .orderBy("count", ascending=False)

# Prikazivanje rezultata
result.show(10, truncate=False)

#svi sto nemaju solo pjesme postavljeni default na 0 te su oni manje popularni u solo pjesmama


+----------------------------------------+-----+------------------+-------------------+---------------------------+---------------------+
|artist_pair_names                       |count|avg_popularity    |less_popular_artist|less_popular_avg_popularity|popularity_difference|
+----------------------------------------+-----+------------------+-------------------+---------------------------+---------------------+
|Bad Bunny & J Balvin                    |131  |14.435114503816793|J Balvin           |7.375                      |7.060114503816793    |
|Arijit Singh & Pritam                   |115  |55.18260869565217 |Pritam             |43.0                       |12.18260869565217    |
|Drifting Cowboys & Hank Williams        |105  |21.885714285714286|Drifting Cowboys   |0.0                        |21.885714285714286   |
|Bad Bunny & Jhayco                      |93   |12.204301075268818|Jhayco             |0.43548387096774194        |11.768817204301076   |
|Bad Bunny & Daddy Yankee         

## #2
Pronađite “breakthrough” pjesme (popularnost >80 u albumima u prosjeku <50) i analizirajte razliku
njihovih audio karakteristika (energija, plesnost, valencija) u odnosu na prosječne karakteristike u ostalim
pjesmama unutar albuma.

In [13]:



# 1. Izračunavamo prosečnu popularnost za svaki album
album_avg_popularity = df.groupBy("album_name").agg(F.avg("popularity").alias("album_avg_popularity"))

# 2. Filtriramo pesme sa popularnošću > 80
breakthrough_songs = df.filter(F.col("popularity") > 80)

# 3. Spajamo sa tabelom albuma kako bismo dobili prosečnu popularnost za svaki album
breakthrough_songs_with_avg = breakthrough_songs.join(album_avg_popularity, on="album_name")

# 4. Filtriramo pesme koje su na albumima sa prosečnom popularnošću < 50
breakthrough_songs_filtered = breakthrough_songs_with_avg.filter(F.col("album_avg_popularity") < 50)

# 5. Izračunavanje prosečnih karakteristika (energija, plesnost, valencija) za pesme unutar svakog albuma
album_audio_features = df.groupBy("album_name") \
    .agg(
        F.avg("energy").alias("avg_energy"),
        F.avg("danceability").alias("avg_danceability"),
        F.avg("valence").alias("avg_valence")
    )

# 6. Spajanje "breakthrough" pesama sa prosečnim audio karakteristikama albuma
breakthrough_with_audio_features = breakthrough_songs_filtered.join(album_audio_features, on="album_name")

# 7. Izračunavanje razlike u audio karakteristikama između "breakthrough" pesama i prosečnih karakteristika albuma
breakthrough_with_audio_features = breakthrough_with_audio_features.withColumn("energy_diff", 
    F.col("energy") - F.col("avg_energy")) \
    .withColumn("danceability_diff", 
    F.col("danceability") - F.col("avg_danceability")) \
    .withColumn("valence_diff", 
    F.col("valence") - F.col("avg_valence"))

# 8. Prikazivanje rezultata
breakthrough_with_audio_features.select("track_name", "album_name", "popularity", 
                                        "avg_energy", "avg_danceability", "avg_valence", 
                                        "energy", "danceability", "valence",
                                        "energy_diff", "danceability_diff", "valence_diff") \
    .show(10, truncate=False)


+-----------------------------------+-------------+----------+------------------+------------------+-------------------+------+------------+-------+--------------------+---------------------+--------------------+
|track_name                         |album_name   |popularity|avg_energy        |avg_danceability  |avg_valence        |energy|danceability|valence|energy_diff         |danceability_diff    |valence_diff        |
+-----------------------------------+-------------+----------+------------------+------------------+-------------------+------+------------+-------+--------------------+---------------------+--------------------+
|Hold On                            |Hold On      |82        |0.6633333333333333|0.5808333333333332|0.24263333333333334|0.443 |0.618       |0.167  |-0.22033333333333333|0.03716666666666679  |-0.07563333333333333|
|Miss You                           |Miss You     |87        |0.6603333333333333|0.5625            |0.2697             |0.742 |0.587       |0.199  |

## #3

Za 5 najpopularnijih žanrova, utvrdite da li postoje optimalni rasponi tempa "sweet spot" upoređujući
prosječnu popularnost pjesama unutar različitih tempa (npr. <100 BPM, 100-120 BPM, >120 BPM).

In [ ]:
genre_avg_popularity = df.groupBy("track_genre").agg(F.avg("popularity").alias("avg_popularity"))

top5_popularity_genres = genre_avg_popularity.orderBy(F.desc("avg_popularity")).limit(5)

tempo_range = df.withColumn(
    "tempo_range",
    F.when(F.col("tempo") < 100, "<100 BPM") \
    .when(F.col("tempo") > 120, ">120 BPM") \
    .otherwise("100-120 BPM")
)

analysis_df = tempo_range.join(top5_popularity_genres.select("track_genre"),on="track_genre",how="inner") \
                         .groupBy("track_genre", "tempo_range").agg(F.avg("popularity").alias("avg_popularity"))

analysis_df.orderBy("track_genre","avg_popularity").show(truncate=False)

+-----------+-----------+------------------+
|track_genre|bpm_range  |avg_popularity    |
+-----------+-----------+------------------+
|pop-film   |<100 BPM   |58.56456456456456 |
|pop-film   |100-120 BPM|59.86363636363637 |
|pop-film   |>120 BPM   |59.51529411764706 |
|k-pop      |<100 BPM   |55.255319148936174|
|k-pop      |100-120 BPM|55.23076923076923 |
|k-pop      |>120 BPM   |58.851528384279476|
|chill      |<100 BPM   |53.07871720116618 |
|chill      |100-120 BPM|53.78902953586498 |
|chill      |>120 BPM   |54.04047619047619 |
|sad        |<100 BPM   |50.729559748427675|
|sad        |100-120 BPM|55.81818181818182 |
|sad        |>120 BPM   |51.87662337662338 |
|grunge     |<100 BPM   |46.35748792270532 |
|grunge     |100-120 BPM|52.86206896551724 |
|grunge     |>120 BPM   |49.60508474576271 |
+-----------+-----------+------------------+



## #4
Uporedite korelaciju između eksplicitnog sadržaja i popularnosti u različitim žanrovima kako biste
utvrdili u kojim je žanrovima eksplicitni sadržaj pozitivan ili negativan faktor za uspjeh pjesme.


In [22]:
# Da bi se mogla koristiti F.corr, jer ne moze sa boolean vrijednostima, mora se pretvoriti u brojeve 0 ili 1
df_sa_explicit_int = df.withColumn("explicit_num", F.when(F.col("explicit") == True, 1).otherwise(0))

rez_korelacije = df_sa_explicit_int.groupBy("track_genre") \
    .agg(
        F.corr("explicit_num", "popularity").alias("korelacija_explicit_popularity")
    )

# Dobijao sam neke NULL vrijednosti, vjerovatno kad nema dovoljno podataka za korelaciju, pa da ih izbjegnem u drugoj tabeli dropam NULL vrijednosti
rez_korelacije = rez_korelacije.na.drop(subset=["korelacija_explicit_popularity"])

rez_korelacije.orderBy(F.desc("korelacija_explicit_popularity")).show(5,truncate=False)
rez_korelacije.orderBy("korelacija_explicit_popularity").show(5,truncate=False)

+-----------+------------------------------+
|track_genre|korelacija_explicit_popularity|
+-----------+------------------------------+
|soul       |0.3655483270888596            |
|house      |0.30992433238428124           |
|groove     |0.30458368188166235           |
|funk       |0.2981659986401298            |
|emo        |0.2811071472943843            |
+-----------+------------------------------+
only showing top 5 rows



+-----------+------------------------------+
|track_genre|korelacija_explicit_popularity|
+-----------+------------------------------+
|hip-hop    |-0.38666863266040846          |
|french     |-0.23838715445624745          |
|indie      |-0.18037233012706097          |
|acoustic   |-0.17193264328187755          |
|garage     |-0.13750323081098528          |
+-----------+------------------------------+
only showing top 5 rows



## #5
Za 10 najdužih pjesama sa visokom ocjenom plesivosti (>0,8), uporedite njihov komercijalni uspjeh
(popularnost) sa prosječnom popularnošću pjesama u istom žanru kako biste utvrdili da li duže trajanje
utiče na uspjeh.

In [25]:
high_danceability_pjesme = df.filter(F.col("danceability") > 0.8)

longest_high_danceability_pjesme = high_danceability_pjesme.orderBy(F.desc("duration_ms")).limit(10)

genre_avg_popularity = df.groupBy("track_genre").agg(F.avg("popularity").alias("avg_genre_popularity"))

longest_songs_with_genre_popularity = longest_high_danceability_pjesme.join(genre_avg_popularity, on="track_genre")

res = longest_songs_with_genre_popularity.withColumn(
    "popularity_razlika", 
    F.col("popularity") - F.col("avg_genre_popularity")
)

res.select("track_name", "duration_ms", "track_genre", "popularity", "avg_genre_popularity", "popularity_razlika") \
    .orderBy(F.desc("duration_ms")).show(truncate=False)

+--------------------------------------------+-----------+--------------+----------+--------------------+-------------------+
|track_name                                  |duration_ms|track_genre   |popularity|avg_genre_popularity|popularity_razlika |
+--------------------------------------------+-----------+--------------+----------+--------------------+-------------------+
|House of Om - Mark Farina - Continuous Mix  |4447520    |chicago-house |11        |12.33366733466934   |-1.3336673346693395|
|Live In Tokyo - Continuous Mix              |4339826    |chicago-house |11        |12.33366733466934   |-1.3336673346693395|
|Greenhouse Construction                     |4334721    |chicago-house |12        |12.33366733466934   |-0.3336673346693395|
|Reggae Dancehall Riddim: Nanny Goat         |2308806    |dancehall     |27        |34.95527156549521   |-7.9552715654952095|
|Various Mixed KDJ Rarities                  |1025706    |detroit-techno|5         |11.130753138075313  |-6.1307531380

## #6
Analizirajte da li eksplicitne i neeksplicitne pjesme pokazuju različite obrasce u valentnosti (muzička
pozitivnost) i utvrdite da li je ovaj odnos dosljedan na svim razinama popularnosti (posmatrati opsege
popularnosti u veličini od 0.1)

In [ ]:
# Ako sam shvatio dobro treba kreirati po veličini od 0.1*popularnosti_max, jer nema smisla popularnost_range da bude 0-0.1, jer je popularnost int
df_popularity_range = df.withColumn(
    "popularity_range",
    (F.col("popularity") / 10).cast("int") * 10  # Kreairanje opsega: 0-10, 10-20, 20-30 itd...
)

explicit_pjesme = df_popularity_range.filter(F.col("explicit") == True)
non_explicit_pjesme = df_popularity_range.filter(F.col("explicit") == False)

explicit_valence = explicit_pjesme.groupBy("popularity_range").agg(F.avg("valence").alias("avg_valence_explicit"))

non_explicit_valence = non_explicit_pjesme.groupBy("popularity_range").agg(F.avg("valence").alias("avg_valence_non_explicit"))

comparison_result = explicit_valence.join(
    non_explicit_valence, on="popularity_range", how="outer"
)

comparison_result = comparison_result.withColumn("popularity_range",F.concat(
        F.lit("["), F.col("popularity_range"),F.lit("-"), F.col("popularity_range").cast("int") + 10, F.lit(")")
    ))

comparison_result.show(truncate=False)

+----------------+--------------------+------------------------+
|popularity_range|avg_valence_explicit|avg_valence_non_explicit|
+----------------+--------------------+------------------------+
|0               |0.5106948996201834  |0.5058986723163853      |
|10              |0.3365437125748502  |0.44451661197238884     |
|20              |0.44598182825484783 |0.4693521155582299      |
|30              |0.4945071213640921  |0.5102801832148774      |
|40              |0.48520854037267075 |0.46349948912558503     |
|50              |0.44641345740873306 |0.4202858229388501      |
|60              |0.4715231364956437  |0.4691525357701096      |
|70              |0.5177853288364247  |0.4993048151169111      |
|80              |0.49790389105058364 |0.5274767139479906      |
|90              |0.43287878787878775 |0.4461253968253968      |
|100             |NULL                |0.238                   |
+----------------+--------------------+------------------------+



## #7

Za umjetnike s najdosljednijom popularnošću (najniža standardna devijacija), analizirajte njihovu
žanrovsku raznolikost kako biste utvrdili da li je dosljednost povezana sa specijalizacijom za određene
žanrove.


In [ ]:
artist_stddev_pop = df.groupBy("artists").agg(F.stddev("popularity").alias("stddev_popularity"))

artist_genres_cnt = df.groupBy("artists").agg(F.countDistinct("track_genre").alias("genre_count"))

artist_stats = artist_stddev_pop.join(artist_genres_cnt, on="artists").filter(F.col("stddev_popularity")>=0)
artist_stats=artist_stats.filter(F.col("genre_count")>0) # U slucaju da treba analizirati pjevace sa vise zanrova, jer ovako ima ih sa 1 pjesmom itd

artist_stats.select("artists", "stddev_popularity", "genre_count").orderBy("stddev_popularity").show(10,truncate=False)

+----------------------------+-----------------+-----------+
|artists                     |stddev_popularity|genre_count|
+----------------------------+-----------------+-----------+
|Ashley McBryde              |0.0              |1          |
|Tasha Cobbs Leonard         |0.0              |1          |
|Joy Oladokun                |0.0              |1          |
|Jaime Guardia               |0.0              |1          |
|Валерий Агафонов            |0.0              |1          |
|Lollies                     |0.0              |1          |
|Tito "El Bambino"           |0.0              |1          |
|Krewella;Nucleya            |0.0              |1          |
|Las Almas Nuevas            |0.0              |1          |
|Elvis Costello;The Imposters|0.0              |1          |
+----------------------------+-----------------+-----------+
only showing top 10 rows



## #8
 U žanrovima sa visokim akustičnim i instrumentalnim vrijednostima (>0,8), uporedite njihovu
prosječnu popularnost sa vokalno teškim žanrovima kako biste utvrdili da li instrumentalni fokus utiče na
komercijalni uspjeh.

Vokalno teski zanrovi su obrnuto od ovih sa visokim akusticnim i instrumentalnim vrijednostima (vokalno lako zanrovi)

In [ ]:
vocal_light_pjesme = df.filter((F.col("acousticness") > 0.8) & (F.col("instrumentalness") > 0.8))
vocal_heavy_pjesme = df.filter((F.col("acousticness") <= 0.2) & (F.col("instrumentalness") <= 0.2))

# Prosjecna popularnost
vocal_light_popularity = vocal_light_pjesme.groupBy("track_genre").agg(F.avg("popularity").alias("avg_popularnost_vokalno_lakih_zanrova"))
vocal_heavy_popularity = vocal_heavy_pjesme.groupBy("track_genre").agg(F.avg("popularity").alias("avg_popularnost_vokalno_teskih_zanrova"))

# Spajanje na osnovu zanra, outer - bice prisutne i vrijednosti koje se ne nalaze u nekoj grupi
comp_res = vocal_light_popularity.join(
    vocal_heavy_popularity, on="track_genre", how="outer"
)

comp_res.orderBy(F.desc("avg_popularnost_vokalno_lakih_zanrova")).show(truncate=False)

+-----------------+-------------------------------------+--------------------------------------+
|track_genre      |avg_popularnost_vokalno_lakih_zanrova|avg_popularnost_vokalno_teskih_zanrova|
+-----------------+-------------------------------------+--------------------------------------+
|indie-pop        |76.5                                 |36.43835616438356                     |
|emo              |64.0                                 |47.68729641693811                     |
|blues            |59.0                                 |41.3343653250774                      |
|folk             |56.75                                |36.265116279069765                    |
|country          |56.0                                 |15.963302752293577                    |
|turkish          |51.4                                 |40.4141689373297                      |
|sad              |51.142857142857146                   |52.16352201257862                     |
|j-rock           |51.0       

Zaključak na osnovu tabele: Za neke zanrove instrumentalni fokus utiče na uspjeh(indie-pop,blues,country...), a na neke ne(chill,electronic...).

## Slaviša Čovakušić 1127/22